It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [23]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

newindex=np.arange(1,df.From_To.count()+1)
newindex
df.set_index(newindex, inplace=True)

for i in np.arange(1,df.From_To.count()+1):
    if pd.isnull(df.FlightNumber.loc[i,]):
        df.loc[i,'FlightNumber'] = df.FlightNumber.loc[i-1] + 10

df['FlightNumber'].astype(int)

1    10045
2    10055
3    10065
4    10075
5    10085
Name: FlightNumber, dtype: int32

2.The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [24]:
temporarydf = df.copy()
temporarydf[['From','To']] = temporarydf.From_To.str.split("_",expand=True)
temporarydf

,From_To,FlightNumber,RecentDelays,Airline,From,To
1,LoNDon_paris,10045.0,"[23, 47]",KLM(!),LoNDon,paris
2,MAdrid_miLAN,10055.0,[],<Air France> (12),MAdrid,miLAN
3,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
4,Budapest_PaRis,10075.0,[13],12. Air France,Budapest,PaRis
5,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air""",Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [25]:
temporarydf.From = temporarydf.From.str.capitalize()
temporarydf.To = temporarydf.To.str.capitalize()
temporarydf.From_To = temporarydf.From_To.str.capitalize()
temporarydf

,From_To,FlightNumber,RecentDelays,Airline,From,To
1,London_paris,10045.0,"[23, 47]",KLM(!),London,Paris
2,Madrid_milan,10055.0,[],<Air France> (12),Madrid,Milan
3,London_stockholm,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
4,Budapest_paris,10075.0,[13],12. Air France,Budapest,Paris
5,Brussels_london,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [26]:
df.drop('From_To',axis=1, inplace = True)
df['From_To'] = temporarydf['From_To']
df

,FlightNumber,RecentDelays,Airline,From_To
1,10045.0,"[23, 47]",KLM(!),London_paris
2,10055.0,[],<Air France> (12),Madrid_milan
3,10065.0,"[24, 43, 87]",(British Airways. ),London_stockholm
4,10075.0,[13],12. Air France,Budapest_paris
5,10085.0,"[67, 32]","""Swiss Air""",Brussels_london


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [31]:
df3 = pd.DataFrame(df['RecentDelays'].values.tolist())
max_length = df3.shape[1]

col_dict={}
for k in range(0, max_length):
    col_dict[k] = 'Delay'+ '_' + str(k+1)

col_dict

df4 = df3.rename(col_dict, axis =1)
df4
    

new_frame = [df,df4]

new_df = pd.concat(new_frame, axis =1)
new_df

df5 = new_df.drop('RecentDelays', axis =1)
df5

,FlightNumber,Airline,From_To,Delay_1,Delay_2,Delay_3
0,NaN,NaN,NaN,23.0,47.0,NaN
1,10045.0,KLM(!),London_paris,NaN,NaN,NaN
2,10055.0,<Air France> (12),Madrid_milan,24.0,43.0,87.0
3,10065.0,(British Airways. ),London_stockholm,13.0,NaN,NaN
4,10075.0,12. Air France,Budapest_paris,67.0,32.0,NaN
5,10085.0,"""Swiss Air""",Brussels_london,NaN,NaN,NaN
